In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from sklearn.preprocessing import StandardScaler
from scipy.interpolate import interp1d

from don import DeepONet_Model


In [2]:
def preprocess(dataset, m, npoints_output, is_test=False):
    p = dataset['del_p']
    t = dataset['t']
    r = dataset['R']

    P = interp1d(t, p, kind='cubic')
    R = interp1d(t, r, kind='cubic')

    t_min = 0
    t_max = 5 * 10**-4

    X_func = P(np.linspace(t_min, t_max, m)) #[1500, m]
    X_loc  = np.linspace(t_min, t_max, npoints_output)[:, None] #[npoints_output,1]
    y      = R(np.ravel(X_loc)) #[1500, npoints_output]

    if is_test:
        X_func = X_func[:50]
        y = y[:50]

    return X_func, X_loc, y

def normalize(X_func, X_loc, y, Par):
    X_func = (X_func - Par['p_mean'])/Par['p_std']
    X_loc  = (X_loc - np.min(X_loc))/(np.max(X_loc) - np.min(X_loc))
    y = (y - Par['r_mean'])/Par['r_std']

    return X_func.astype(np.float32), X_loc.astype(np.float32), y.astype(np.float32)


In [12]:
def main():
    Par = {}
    Par['address'] = 'don'

    train_dataset = np.load('../data/0.1/res_1000.npz')
    test_dataset = np.load('../data/0.08/res_1000.npz')

    m = 20
    npoints_output = 200

    X_func_train, X_loc_train, y_train = preprocess(train_dataset, m, npoints_output)
    X_func_test, X_loc_test, y_test = preprocess(test_dataset, m, npoints_output, is_test=True)

    Par['p_mean'] = np.mean(X_func_train)
    Par['p_std']  = np.std(X_func_train)

    Par['r_mean'] = np.mean(y_train)
    Par['r_std']  = np.std(y_train)
    
    plt.plot(range(len(X_func_train[0])), X_func_train[0])
    plt.show()
    
    X_func_train, X_loc_train, y_train = normalize(X_func_train, X_loc_train, y_train, Par)
    X_func_test, X_loc_test, y_test    = normalize(X_func_test, X_loc_test, y_test, Par)

    print('X_func_train: ', X_func_train.shape, '\nX_loc_train: ', X_loc_train.shape, '\ny_train: ', y_train.shape)
    print('X_func_test: ', X_func_test.shape, '\nX_loc_test: ', X_loc_test.shape, '\ny_test: ', y_test.shape)


    don_model = DeepONet_Model(Par)
    don_model_address = 'don/model_100000'
    don_model.load_weights(don_model_address)
    
    for i in [0.07, 0.08, 0.09, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
        print('\nGRF(l={:.1})'.format(i))
        print('-'*20)
        
        address = '../data/'+str(i)+'/res_1000.npz'
        test_dataset = np.load(address)
        
        m = 20
        npoints_output = 500

        X_func_test, X_loc_test, y_test = preprocess(test_dataset, m, npoints_output, is_test=True)
        X_func_test, X_loc_test, y_test = normalize(X_func_test[:1], X_loc_test, y_test[:1], Par)

        y_pred = don_model(X_func_test, X_loc_test)
        mse = np.mean((y_pred - y_test)**2)
        print('MSE: {:.4e}'.format(mse))
        
        y_pred = y_pred*Par['r_std'] + Par['r_mean']
        y_test = y_test*Par['r_std'] + Par['r_mean']
        
        plt.figure(figsize=(10,10))
        plt.plot(np.ravel(X_loc_test* 5*10**-4), np.ravel(y_pred), label='DeepONet')
        plt.plot(np.ravel(X_loc_test* 5*10**-4), np.ravel(y_test), label='truth')
        plt.xlabel('time (s)', fontsize=18)
        plt.ylabel('R (m)', fontsize=18)
        plt.legend(fontsize=16)
        plt.title('GRF(l={:.1})'.format(i), fontsize=22)
        plt.savefig('don/predictions/'+str(i)+'.png')
        plt.close()
        
    
main()


/var/folders/l6/c71cysms70502lf8y911rpkw0000gp/T/ipykernel_9880/3826149143.py:21: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


X_func_train:  (1500, 20) 
X_loc_train:  (200, 1) 
y_train:  (1500, 200)
X_func_test:  (50, 20) 
X_loc_test:  (200, 1) 
y_test:  (50, 200)

GRF(l=0.07)
--------------------
MSE: 6.7249e-03

GRF(l=0.08)
--------------------
MSE: 4.1967e-03

GRF(l=0.09)
--------------------
MSE: 3.4370e-03

GRF(l=0.1)
--------------------
MSE: 3.1379e-03

GRF(l=0.2)
--------------------
MSE: 2.6408e-03

GRF(l=0.3)
--------------------
MSE: 2.2370e-03

GRF(l=0.4)
--------------------
MSE: 1.9579e-03

GRF(l=0.5)
--------------------
MSE: 1.9079e-03

GRF(l=0.6)
--------------------
MSE: 1.9480e-03

GRF(l=0.7)
--------------------
MSE: 1.9973e-03

GRF(l=0.8)
--------------------
MSE: 2.0367e-03


In [26]:
train_dataset = np.load('../data/0.07/res_1000.npz')


m = 1000
npoints_output = 200

X_func_train, X_loc_train, y_train = preprocess(train_dataset, m, npoints_output)

plt.close()
plt.plot(range(len(X_func_train[0])), X_func_train[0])
plt.savefig('temp3.png')
plt.close()